
Exercício 2

Este exercício é dirigido à prova de correção do algoritmo estendido de Euclides

    1. Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função gcd é gcd(a,b) = min{r > 0 | E [s,t] . r = a * s + b * t}.
    2. Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando assert .
    3. Construa codificações do programa LPA através de transformadores de predicados: “weakest pre-condition” e “strongest post-condition”. 
    4. Prove a correção  do programa LPA em ambas as codificações.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT,BVType
from pysmt.logics import QF_NIA
import itertools

1.

In [2]:
a = Symbol('a',INT)
b = Symbol('b',INT)
r = Symbol('r',INT)

n = Symbol('n',INT)
r_ = Symbol('r_',INT)
s = Symbol('s',INT)
s_ = Symbol('s_',INT)
t = Symbol('t',INT)
t_ = Symbol('t_',INT)
q = Symbol('q' ,INT)

inv = And(GT(r,Int(0)), LT(r,n), Equals(r, Plus(Times(a,s), Times(b,t))))

pos = And(Not(And(LE(r_, r), Equals(Plus(Times(a, s_), Times(b, t_)), r_))), inv)

print(pos)

((! ((r_ <= r) & ((... + ...) = r_))) & ((0 < r) & (r < n) & (r = ((... * ...) + (... * ...)))))


2. LPA & havoc

w ≡ {assume b; S; W} || {^assume b} 

S = q = r div r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'  
w = {assume r' != 0; q = r div r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; W} || {assume not r' != 0}  

In [3]:
def prove(f):
    with Solver(name="z3") as solver:
        solver.add_assertion(Not(f))
        if solver.solve():
            print("Proved")
        else:
            print("Failed to prove")

In [4]:
func = Implies(And(Not(Equals(r, Int(0))),inv),substitute(
    substitute(
        substitute(
            substitute(
                substitute(
                    substitute(inv,
                               {t_:t - q * t}),
                    {t:t_}),
                {s_:s - q * s_}),
            {s:s_}),
        {r_:r - q * r_}),
    {r:r_}))

havoc = ForAll([r, r_, s, s_, t, t_], func)
axioms = And(havoc, And(Equals(r, a), Equals(r_, b), Equals(s, Int(1)), Equals(s_, Int(0)), Equals(t, Int(0)), Equals(t_, Int(1))))

prove(havoc)
prove(axioms)

Proved
Proved


## Codificar em SMT e provar a correção
Iremos provar a correção do ciclo usando havoc.
Na metodologia havoc, o ciclo (while b do{θ}), com anotação de invariante θ é transformado num fluxo não iterativo da seguinte forma

    assert θ ; C ; havoc x ; ( (assume b ∧ θ ; assert θ ; assume False) || assume ¬b ∧ θ )

onde x
    representa as variáveis atribuídas em C.

Iremos então traduzir o triplo de Hoare {ϕ}while b do{θ}{ψ}, da seguinte forma, de modo a garantir as propriedades de
"inicialização", "preservação" e "utilidade" do invariante θ

    [ assume ϕ ; C ; assert θ ; havoc x ; ( (assume b ∧ θ ; assert θ ; assume False) || assume ¬b ∧ θ ) ; assert ψ ]

=

    ϕ → [C , θ ∧ ∀x. ((b ∧ θ → assert θ) ∧ (¬b ∧ θ → ψ))]


Em primeiro lugar, iremos traduzir o programa para a linguagem de fluxos com havoc:  
assume pre;  
assert invariante;  
havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t';    
((assume b and invariante; assert invariante; assume False) || assume (not b) and invariante);  
assert pos  

Em seguida iremos calcular a denotação lógica deste programa de fluxos pela WPC, calculada pelas seguintes regras:  
[assume ϕ] = True  
[assert ϕ] = ϕ  
[x = e] = True  

[assume pre; assert invariante; havoc r, r', s, s', t, t'; ((assume (b and invariante); assert invariante; assume False) || assume ((not b) and invariante)); assert pos]

=

[assume a > 0 and b > 0 and a < N and b < N; havoc f; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1;]  

=

(a > 0 and b > 0 and a < N and b < N) -> r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1;  
(r= a*s+b*t and r>0 and r < N) and forall (r, r', s, s', t, t') . 
((not(r' == 0) and (r= a*s+b*t and r>0 and r < N) ->  [
q = r div r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t';   
assert inv])  
and ( r' == 0 and (r= a*s+b*t and r>0 and r < N) -> ((r= a*s+b*t and r>0 and r < N),not(r_<=r and r_= a*s_+b*t_))))]  

=

(a > 0 and b > 0 and a < N and b < N) -> [forall (r, r', s, s', t, t') . 
((not(r' == 0) and (r= a*s+b*t and r>0 and r < N) ->  [
q = r div r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t';   
assert inv]); ([r = a]) [r' = b]) [s = 1]) [s' = 0]) [t = 0]) [t' = 1]))]

Finalmente, passaremos à prova da correção do programa, usando o Z3.

## WPC

[assume a > 0 and b > 0 and a < N and b < N; havoc f; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1;]  

=

(a > 0 and b > 0 and a < N and b < N) -> [havoc x and ([r = a]) [r' = b]) [s = 1]) [s' = 0]) [t = 0]) [t' = 1])), pos]

In [5]:
WPC = substitute(
        substitute(
            substitute(
                substitute(
                    substitute(
                        substitute(pos,{t_:Int(1)}),
                        {t:Int(0)}),
                    {s_:Int(0)}),
                {s:Int(1)}),
            {r_:b}),
        {r:a})

## SPC

[assume a > 0 and b > 0 and a < N and b < N; havoc x; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; pos]  

= [assume a > 0 and b > 0 and a < N and b < N; havoc x; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1;]  -> ((r = a * s + b * t and r > 0 and r < N), not(r_ <= r and r_= a * s_+ b * t_))

= (a > 0 and b > 0 and a < N and b < N and havoc x and r = a and r' = b and s = 1 and s' = 0 and t = 0 and t' = 1) -> ((r = a * s + b * t and r > 0 and r < N), not(r_ <= r and r_= a * s_+ b * t_)))

In [6]:
SPC = And(Equals(r, a),
          Equals(r_, b),
          Equals(s, Int(1)),
          Equals(s_, Int(0)),
          Equals(t, Int(0)),
          Equals(t_, Int(1)),
         )

In [7]:
pre = (a > 0) and (b > 0) and (a < n) and (b < n)

WPC_final = Implies(pre, 
                    And(WPC,
                       axioms,
                       pos))

SPC_final = Implies(And(pre,
                    axioms,
                    SPC),
                    pos)

print('WPC')
prove(WPC_final)
print('SPC')
prove(SPC_final)

WPC
Proved
SPC
Proved
